## Imports


In [1]:
import pandas as pd
import numpy as np
import nltk
import os
import re
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from scipy.stats import ks_2samp
from IPython.core.display import display, HTML

C:\Users\krusic\AppData\Local\Temp\ipykernel_24984\2250226892.py:10: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
tqdm.pandas()

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\krusic\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.


True

## Configuration 

*dictionary_dir:* The path to the directory containing the dictionaries. If you use the default directory structure, you can use `dictionaries/manual/` for dictionaries containing only manual annotated words and `dictionaries/computational/` for dictionaries containing manual and automatically extended words. Please make sure to use a '/' (slash) in the end. For example: `path/to/dictionaries/`.

*corpus:* The full path to the filename of the pandas DataFrame which contains our corpus of narrative forms. 

*output:* Path to directory where all resulting files are stored.

In [78]:
corpus = "input/spectators_es_narratives_selected.p"
output = "output/files/"

In [50]:
dictionary_manual = "input/dictionaries/manual/"
dictionaryCorr = "input/dictionaries/computational_corrected/"
dictionaryComp = "input/dictionaries/computational/"

## Sentiment Analysis using pysentimiento (https://github.com/pysentimiento; Perez et al, 2019)

In [22]:
!pip install pysentimiento

   ---------------------------------------- 0.0/324.4 kB ? eta -:--:--
   --------------------------------------- 324.4/324.4 kB 10.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/286.0 kB ? eta -:--:--
   --------------------------------------- 286.0/286.0 kB 18.4 MB/s eta 0:00:00
  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.4.2
    Uninstalling safetensors-0.4.2:
      Successfully uninstalled safetensors-0.4.2


In [26]:
from pysentimiento import create_analyzer

In [27]:
df = pd.read_pickle(dataframe_filename)

df.head()

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text
filename,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,En tànto que el Duende espèra á ver si purga e...
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabu..."
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado po..."
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.4.txt,o:mws-112-841,FP,FR.4,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"A Patiño. A el Nabuco de estos tiempos,á el qu..."
1761-06-09_El-Duende-especulativo-sobre-la-vida-civil_Juan-Antonio-Mercadal-_Vol-1_Nr-01_093-235_FR.10.txt,o:mws-093-235,EX,FR.10,El-Duende-especulativo-sobre-la-vida-civil,Juan Antonio Mercadal [Francisco Mariano Nipho...,1761-06-09,1-01,"El chistoso Svvift pretendìa, que los Ingleses..."


In [28]:
df

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text
filename,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,En tànto que el Duende espèra á ver si purga e...
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabu..."
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado po..."
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.4.txt,o:mws-112-841,FP,FR.4,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"A Patiño. A el Nabuco de estos tiempos,á el qu..."
1761-06-09_El-Duende-especulativo-sobre-la-vida-civil_Juan-Antonio-Mercadal-_Vol-1_Nr-01_093-235_FR.10.txt,o:mws-093-235,EX,FR.10,El-Duende-especulativo-sobre-la-vida-civil,Juan Antonio Mercadal [Francisco Mariano Nipho...,1761-06-09,1-01,"El chistoso Svvift pretendìa, que los Ingleses..."
...,...,...,...,...,...,...,...,...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.11.txt,o:mws.8083,FP,FR.11,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,“Mira. . . ; pero yo no podré mostrarte en est...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.3.txt,o:mws.8083,AE,FR.3,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,La más violenta devanadera en todo su ejercici...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.5.txt,o:mws.8083,TR,FR.5,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,Pero apenas el perezoso Morfeo había embargado...


we want to add the nr of tokens present in the text column, so 

In [29]:

df['text'] = df['text'].astype('string')
print(df.dtypes)

issue pid                   object
narrative form              object
xmlid                       object
periodical title            object
author                      object
year                        object
issue number                object
text                string[python]
dtype: object


In [30]:
df['new_text'] = df['text'].apply(lambda x: x.lower())
df ['new_text']= df['new_text'].replace('\d+', '', regex = True)
df['new_text'] = df['new_text'].replace('[^\w\s\+]', '', regex = True)
df['new_column'] = df.new_text.apply(lambda x: len(str(x).split(' ')))
df

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text,new_text,new_column
filename,,,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,En tànto que el Duende espèra á ver si purga e...,en tànto que el duende espèra á ver si purga e...,274
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabu...",cuento de cuentos historía sin verdad i fabula...,696
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado po...",vele aquí el cuento acabadoaun que cortado por...,36
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.4.txt,o:mws-112-841,FP,FR.4,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"A Patiño. A el Nabuco de estos tiempos,á el qu...",a patiño a el nabuco de estos tiemposá el que ...,25
1761-06-09_El-Duende-especulativo-sobre-la-vida-civil_Juan-Antonio-Mercadal-_Vol-1_Nr-01_093-235_FR.10.txt,o:mws-093-235,EX,FR.10,El-Duende-especulativo-sobre-la-vida-civil,Juan Antonio Mercadal [Francisco Mariano Nipho...,1761-06-09,1-01,"El chistoso Svvift pretendìa, que los Ingleses...",el chistoso svvift pretendìa que los ingleses ...,90
...,...,...,...,...,...,...,...,...,...,...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.11.txt,o:mws.8083,FP,FR.11,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,“Mira. . . ; pero yo no podré mostrarte en est...,mira pero yo no podré mostrarte en este rat...,420
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.3.txt,o:mws.8083,AE,FR.3,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,La más violenta devanadera en todo su ejercici...,la más violenta devanadera en todo su ejercici...,2542
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.5.txt,o:mws.8083,TR,FR.5,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,Pero apenas el perezoso Morfeo había embargado...,pero apenas el perezoso morfeo había embargado...,1992


In [31]:
df = df.drop(['new_text'], axis=1)


In [32]:
df.rename(columns = {'new_column':'nr_tokens'}, inplace = True)
df

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text,nr_tokens
filename,,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,En tànto que el Duende espèra á ver si purga e...,274
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabu...",696
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado po...",36
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.4.txt,o:mws-112-841,FP,FR.4,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"A Patiño. A el Nabuco de estos tiempos,á el qu...",25
1761-06-09_El-Duende-especulativo-sobre-la-vida-civil_Juan-Antonio-Mercadal-_Vol-1_Nr-01_093-235_FR.10.txt,o:mws-093-235,EX,FR.10,El-Duende-especulativo-sobre-la-vida-civil,Juan Antonio Mercadal [Francisco Mariano Nipho...,1761-06-09,1-01,"El chistoso Svvift pretendìa, que los Ingleses...",90
...,...,...,...,...,...,...,...,...,...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.11.txt,o:mws.8083,FP,FR.11,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,“Mira. . . ; pero yo no podré mostrarte en est...,420
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.3.txt,o:mws.8083,AE,FR.3,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,La más violenta devanadera en todo su ejercici...,2542
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.5.txt,o:mws.8083,TR,FR.5,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,Pero apenas el perezoso Morfeo había embargado...,1992


To start conducting the sentiment analysis with pysentimiento, we are calling on the function create_analyzer, which as input takes the task of the analysis as well as the language. Firstly we conduct sentiment analysis on the column "text" after which we conduct emotion analysis by changing the value of task. We store the results in the columns "nf_sentiment_pysentimiento" and "nf_emotion_pysentimiento", respectively.

In [33]:
analyzer = create_analyzer (task='sentiment', lang='es')

config.json:   0%|          | 0.00/925 [00:00<?, ?B/s]

C:\Users\krusic\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\krusic\.cache\huggingface\hub\models--pysentimiento--robertuito-sentiment-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [34]:
df['nf_sentiment_pysentimiento'] = analyzer.predict(df['text'])


Map:   0%|          | 0/3053 [00:00<?, ? examples/s]

In [36]:
emotion_analyzer = create_analyzer(task="emotion", lang="es")

config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

C:\Users\krusic\AppData\Local\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\krusic\.cache\huggingface\hub\models--pysentimiento--robertuito-emotion-analysis. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/435M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/384 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [37]:
df['nf_emotion_pysentimiento'] = emotion_analyzer.predict(df['text'])


Map:   0%|          | 0/3053 [00:00<?, ? examples/s]

In [40]:
df

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text,nr_tokens,nf_sentiment_pysentimiento,nf_emotion_pysentimiento
filename,,,,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,En tànto que el Duende espèra á ver si purga e...,274,"AnalyzerOutput(output=NEU, probas={NEU: 0.552,...","AnalyzerOutput(output=others, probas={others: ..."
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabu...",696,"AnalyzerOutput(output=NEU, probas={NEU: 0.603,...","AnalyzerOutput(output=anger, probas={anger: 0...."
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado po...",36,"AnalyzerOutput(output=NEU, probas={NEU: 0.524,...","AnalyzerOutput(output=others, probas={others: ..."
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.4.txt,o:mws-112-841,FP,FR.4,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"A Patiño. A el Nabuco de estos tiempos,á el qu...",25,"AnalyzerOutput(output=NEG, probas={NEG: 0.533,...","AnalyzerOutput(output=others, probas={others: ..."
1761-06-09_El-Duende-especulativo-sobre-la-vida-civil_Juan-Antonio-Mercadal-_Vol-1_Nr-01_093-235_FR.10.txt,o:mws-093-235,EX,FR.10,El-Duende-especulativo-sobre-la-vida-civil,Juan Antonio Mercadal [Francisco Mariano Nipho...,1761-06-09,1-01,"El chistoso Svvift pretendìa, que los Ingleses...",90,"AnalyzerOutput(output=NEU, probas={NEU: 0.680,...","AnalyzerOutput(output=others, probas={others: ..."
...,...,...,...,...,...,...,...,...,...,...,...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.11.txt,o:mws.8083,FP,FR.11,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,“Mira. . . ; pero yo no podré mostrarte en est...,420,"AnalyzerOutput(output=NEG, probas={NEG: 0.900,...","AnalyzerOutput(output=anger, probas={anger: 0...."
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.3.txt,o:mws.8083,AE,FR.3,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,La más violenta devanadera en todo su ejercici...,2542,"AnalyzerOutput(output=NEU, probas={NEU: 0.589,...","AnalyzerOutput(output=others, probas={others: ..."
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.5.txt,o:mws.8083,TR,FR.5,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,Pero apenas el perezoso Morfeo había embargado...,1992,"AnalyzerOutput(output=NEG, probas={NEG: 0.625,...","AnalyzerOutput(output=others, probas={others: ..."


# Dictionary Sentiment analysis  on narrative forms

In [72]:
import operator

### Manually created sentiment dictionary

In [52]:
sentiment_dict = {}
with open("{}{}_negative.txt".format(dictionary_manual, language.lower()), "r", encoding="utf-8") as fr:
    sentiment_dict["neg"] = fr.read().splitlines()
with open("{}{}_positive.txt".format(dictionary_manual, language.lower()), "r", encoding="utf-8") as fr:
    sentiment_dict["pos"] = fr.read().splitlines()

print("loaded {} negative words".format(len(sentiment_dict["neg"])))
print("loaded {} positive words".format(len(sentiment_dict["pos"])))

loaded 798 negative words
loaded 681 positive words


### Calculate Sentiments

In [47]:
def compute_sentiment(text):
    tokens = nltk.word_tokenize(text)
    tokens = [t.lower() for t in tokens]
    num_negative = 0
    num_positive = 0
    for nw in sentiment_dict["neg"]:
        num_negative += tokens.count(nw.lower())
    for pw in sentiment_dict["pos"]:
        num_positive += tokens.count(pw.lower())
    try:
        sentiment_score = (num_positive - num_negative) / (num_positive + num_negative)
    except ZeroDivisionError:
        sentiment_score = 0
    return sentiment_score

In [48]:
df["nf_sentiment_lexicon_dispecs"] = df["text"].progress_apply(compute_sentiment)

  0%|          | 0/3053 [00:00<?, ?it/s]

In [49]:

df

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text,nr_tokens,nf_sentiment_pysentimiento,nf_emotion_pysentimiento,nf_sentiment_lexicon_dispecs
filename,,,,,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,En tànto que el Duende espèra á ver si purga e...,274,"AnalyzerOutput(output=NEU, probas={NEU: 0.552,...","AnalyzerOutput(output=others, probas={others: ...",0.466667
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabu...",696,"AnalyzerOutput(output=NEU, probas={NEU: 0.603,...","AnalyzerOutput(output=anger, probas={anger: 0....",0.409091
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado po...",36,"AnalyzerOutput(output=NEU, probas={NEU: 0.524,...","AnalyzerOutput(output=others, probas={others: ...",1.000000
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.4.txt,o:mws-112-841,FP,FR.4,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"A Patiño. A el Nabuco de estos tiempos,á el qu...",25,"AnalyzerOutput(output=NEG, probas={NEG: 0.533,...","AnalyzerOutput(output=others, probas={others: ...",0.333333
1761-06-09_El-Duende-especulativo-sobre-la-vida-civil_Juan-Antonio-Mercadal-_Vol-1_Nr-01_093-235_FR.10.txt,o:mws-093-235,EX,FR.10,El-Duende-especulativo-sobre-la-vida-civil,Juan Antonio Mercadal [Francisco Mariano Nipho...,1761-06-09,1-01,"El chistoso Svvift pretendìa, que los Ingleses...",90,"AnalyzerOutput(output=NEU, probas={NEU: 0.680,...","AnalyzerOutput(output=others, probas={others: ...",0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.11.txt,o:mws.8083,FP,FR.11,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,“Mira. . . ; pero yo no podré mostrarte en est...,420,"AnalyzerOutput(output=NEG, probas={NEG: 0.900,...","AnalyzerOutput(output=anger, probas={anger: 0....",-0.027027
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.3.txt,o:mws.8083,AE,FR.3,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,La más violenta devanadera en todo su ejercici...,2542,"AnalyzerOutput(output=NEU, probas={NEU: 0.589,...","AnalyzerOutput(output=others, probas={others: ...",0.052133
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.5.txt,o:mws.8083,TR,FR.5,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,Pero apenas el perezoso Morfeo había embargado...,1992,"AnalyzerOutput(output=NEG, probas={NEG: 0.625,...","AnalyzerOutput(output=others, probas={others: ...",0.043478


### Computational dictionary

In [53]:
sentiment_dict = {}
with open("{}{}_negative.txt".format(dictionaryComp, language.lower()), "r", encoding="utf-8") as fr:
    sentiment_dict["neg"] = fr.read().splitlines()
with open("{}{}_positive.txt".format(dictionaryComp, language.lower()), "r", encoding="utf-8") as fr:
    sentiment_dict["pos"] = fr.read().splitlines()

print("loaded {} negative words".format(len(sentiment_dict["neg"])))
print("loaded {} positive words".format(len(sentiment_dict["pos"])))

loaded 691 negative words
loaded 1034 positive words


In [54]:
df["nf_sentiment_lexicon_computational"] = df["text"].progress_apply(compute_sentiment)

  0%|          | 0/3053 [00:00<?, ?it/s]

In [55]:
df

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text,nr_tokens,nf_sentiment_pysentimiento,nf_emotion_pysentimiento,nf_sentiment_lexicon_dispecs,nf_sentiment_lexicon_computational
filename,,,,,,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,En tànto que el Duende espèra á ver si purga e...,274,"AnalyzerOutput(output=NEU, probas={NEU: 0.552,...","AnalyzerOutput(output=others, probas={others: ...",0.466667,0.647059
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabu...",696,"AnalyzerOutput(output=NEU, probas={NEU: 0.603,...","AnalyzerOutput(output=anger, probas={anger: 0....",0.409091,0.391304
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado po...",36,"AnalyzerOutput(output=NEU, probas={NEU: 0.524,...","AnalyzerOutput(output=others, probas={others: ...",1.000000,1.000000
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.4.txt,o:mws-112-841,FP,FR.4,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"A Patiño. A el Nabuco de estos tiempos,á el qu...",25,"AnalyzerOutput(output=NEG, probas={NEG: 0.533,...","AnalyzerOutput(output=others, probas={others: ...",0.333333,0.333333
1761-06-09_El-Duende-especulativo-sobre-la-vida-civil_Juan-Antonio-Mercadal-_Vol-1_Nr-01_093-235_FR.10.txt,o:mws-093-235,EX,FR.10,El-Duende-especulativo-sobre-la-vida-civil,Juan Antonio Mercadal [Francisco Mariano Nipho...,1761-06-09,1-01,"El chistoso Svvift pretendìa, que los Ingleses...",90,"AnalyzerOutput(output=NEU, probas={NEU: 0.680,...","AnalyzerOutput(output=others, probas={others: ...",0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.11.txt,o:mws.8083,FP,FR.11,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,“Mira. . . ; pero yo no podré mostrarte en est...,420,"AnalyzerOutput(output=NEG, probas={NEG: 0.900,...","AnalyzerOutput(output=anger, probas={anger: 0....",-0.027027,0.142857
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.3.txt,o:mws.8083,AE,FR.3,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,La más violenta devanadera en todo su ejercici...,2542,"AnalyzerOutput(output=NEU, probas={NEU: 0.589,...","AnalyzerOutput(output=others, probas={others: ...",0.052133,0.082569
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.5.txt,o:mws.8083,TR,FR.5,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,Pero apenas el perezoso Morfeo había embargado...,1992,"AnalyzerOutput(output=NEG, probas={NEG: 0.625,...","AnalyzerOutput(output=others, probas={others: ...",0.043478,0.083333


### Manually created and computationally corrected dictionary

In [56]:
sentiment_dict = {}
with open("{}{}_negative.txt".format(dictionaryCorr, language.lower()), "r", encoding="utf-8") as fr:
    sentiment_dict["neg"] = fr.read().splitlines()
with open("{}{}_positive.txt".format(dictionaryCorr, language.lower()), "r", encoding="utf-8") as fr:
    sentiment_dict["pos"] = fr.read().splitlines()

print("loaded {} negative words".format(len(sentiment_dict["neg"])))
print("loaded {} positive words".format(len(sentiment_dict["pos"])))

loaded 812 negative words
loaded 692 positive words


In [57]:
df["nf_sentiment_lexicon_corrected"] = df["text"].progress_apply(compute_sentiment)

  0%|          | 0/3053 [00:00<?, ?it/s]

In [58]:
df

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text,nr_tokens,nf_sentiment_pysentimiento,nf_emotion_pysentimiento,nf_sentiment_lexicon_dispecs,nf_sentiment_lexicon_computational,nf_sentiment_lexicon_corrected
filename,,,,,,,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,En tànto que el Duende espèra á ver si purga e...,274,"AnalyzerOutput(output=NEU, probas={NEU: 0.552,...","AnalyzerOutput(output=others, probas={others: ...",0.466667,0.647059,0.466667
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabu...",696,"AnalyzerOutput(output=NEU, probas={NEU: 0.603,...","AnalyzerOutput(output=anger, probas={anger: 0....",0.409091,0.391304,0.422222
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado po...",36,"AnalyzerOutput(output=NEU, probas={NEU: 0.524,...","AnalyzerOutput(output=others, probas={others: ...",1.000000,1.000000,1.000000
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.4.txt,o:mws-112-841,FP,FR.4,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"A Patiño. A el Nabuco de estos tiempos,á el qu...",25,"AnalyzerOutput(output=NEG, probas={NEG: 0.533,...","AnalyzerOutput(output=others, probas={others: ...",0.333333,0.333333,0.333333
1761-06-09_El-Duende-especulativo-sobre-la-vida-civil_Juan-Antonio-Mercadal-_Vol-1_Nr-01_093-235_FR.10.txt,o:mws-093-235,EX,FR.10,El-Duende-especulativo-sobre-la-vida-civil,Juan Antonio Mercadal [Francisco Mariano Nipho...,1761-06-09,1-01,"El chistoso Svvift pretendìa, que los Ingleses...",90,"AnalyzerOutput(output=NEU, probas={NEU: 0.680,...","AnalyzerOutput(output=others, probas={others: ...",0.333333,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.11.txt,o:mws.8083,FP,FR.11,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,“Mira. . . ; pero yo no podré mostrarte en est...,420,"AnalyzerOutput(output=NEG, probas={NEG: 0.900,...","AnalyzerOutput(output=anger, probas={anger: 0....",-0.027027,0.142857,-0.027027
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.3.txt,o:mws.8083,AE,FR.3,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,La más violenta devanadera en todo su ejercici...,2542,"AnalyzerOutput(output=NEU, probas={NEU: 0.589,...","AnalyzerOutput(output=others, probas={others: ...",0.052133,0.082569,0.056604
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.5.txt,o:mws.8083,TR,FR.5,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,Pero apenas el perezoso Morfeo había embargado...,1992,"AnalyzerOutput(output=NEG, probas={NEG: 0.625,...","AnalyzerOutput(output=others, probas={others: ...",0.043478,0.083333,0.049383


#  Emotion analysis using the SEL dictionary (Sidorov et al., 2013), annotated with Eckman's basic emotions

In [65]:
new_dictionary_dir = "input/dictionaries/emotion/"

emotion_dict = {}
with open(new_dictionary_dir + "anger.txt", "r", encoding="utf-8") as em:
    emotion_dict["anger"] = em.read().splitlines()
with open(new_dictionary_dir + "disgust.txt", "r", encoding="utf-8") as em:
    emotion_dict["disgust"] = em.read().splitlines()
with open(new_dictionary_dir + "joy.txt", "r", encoding="utf-8") as em:
    emotion_dict["joy"] = em.read().splitlines()
with open(new_dictionary_dir + "fear.txt", "r", encoding="utf-8") as em:
    emotion_dict["fear"] = em.read().splitlines()
with open(new_dictionary_dir + "sadness.txt", "r", encoding="utf-8") as em:
    emotion_dict["sadness"] = em.read().splitlines()
with open(new_dictionary_dir +"surprise.txt", "r", encoding="utf-8") as em:
    emotion_dict["surprise"] = em.read().splitlines()
    

print("loaded {} anger words".format(len(emotion_dict["anger"])))
print("loaded {} disgust words".format(len(emotion_dict["disgust"])))
print("loaded {} joy words".format(len(emotion_dict["joy"])))
print("loaded {} fear words".format(len(emotion_dict["fear"])))
print("loaded {} sadness words".format(len(emotion_dict["sadness"])))
print("loaded {} surprise words".format(len(emotion_dict["surprise"])))

loaded 382 anger words
loaded 209 disgust words
loaded 668 joy words
loaded 211 fear words
loaded 391 sadness words
loaded 175 surprise words


In [69]:
def compute_emotion(text):
    tokens = nltk.word_tokenize(text)
    tokens = [t.lower() for t in tokens]
    num_anger = 0
    num_disgust = 0
    num_joy = 0
    num_fear = 0
    num_sadness = 0
    num_surprise = 0
    
    # Count occurrences of each emotion word in the text
    for anger in emotion_dict["anger"]:
        num_anger += tokens.count(anger.lower())
    for disgust in emotion_dict["disgust"]:
        num_disgust += tokens.count(disgust.lower())
    for joy in emotion_dict["joy"]:
        num_joy += tokens.count(joy.lower())
    for fear in emotion_dict["fear"]:
        num_fear += tokens.count(fear.lower())
    for sadness in emotion_dict["sadness"]:
        num_sadness += tokens.count(sadness.lower())
    for surprise in emotion_dict["surprise"]:
        num_surprise += tokens.count(surprise.lower())

    # Initialize sorted_emotion_score to ensure it's always defined
    sorted_emotion_score = {}

    try:
        emotion_score = {'anger': num_anger, 'joy': num_joy, 'disgust': num_disgust, 'fear': num_fear, 'sadness': num_sadness, 'surprise': num_surprise}
        sorted_emotion_score = dict(sorted(emotion_score.items(), key=operator.itemgetter(1), reverse=True))
    except Exception as e:
        print(f"Something went wrong: {e}")

    # Check if all values are zero (no emotion words found)
    if all(value == 0 for value in sorted_emotion_score.values()):
        return "others"
    else:
        return sorted_emotion_score

In [61]:
def one_emotion(text):
    tokens = nltk.word_tokenize(text)
    tokens = [t.lower() for t in tokens]
    num_anger = 0
    num_disgust = 0
    num_joy = 0
    num_fear = 0
    num_sadness = 0
    num_surprise = 0
    for anger in emotion_dict["anger"]:
        num_anger += tokens.count(anger.lower())
    for disgust in emotion_dict["disgust"]:
        num_disgust += tokens.count(disgust.lower())
    for joy in emotion_dict["joy"]:
        num_joy += tokens.count(joy.lower())
    for fear in emotion_dict["fear"]:
        num_fear += tokens.count(fear.lower())
    for sadness in emotion_dict["sadness"]:
        num_sadness += tokens.count(sadness.lower())
    for surprise in emotion_dict["surprise"]:
        num_surprise += tokens.count(surprise.lower())
    try:
        emotion_score = {'anger' : num_anger,'joy' : num_joy,'disgust' : num_disgust,'fear': num_fear, 'sadness': num_sadness, 'surprise':num_surprise}
        emotion = max(emotion_score, key=emotion_score.get)

    except:
        print("Something went wrong")
    if all(value == 0 for value in emotion_score.values()):
        return ("others")
    else:
        return emotion

In [67]:
df

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text,nr_tokens,nf_sentiment_pysentimiento,nf_emotion_pysentimiento,nf_sentiment_lexicon_dispecs,nf_sentiment_lexicon_computational,nf_sentiment_lexicon_corrected
filename,,,,,,,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,En tànto que el Duende espèra á ver si purga e...,274,"AnalyzerOutput(output=NEU, probas={NEU: 0.552,...","AnalyzerOutput(output=others, probas={others: ...",0.466667,0.647059,0.466667
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabu...",696,"AnalyzerOutput(output=NEU, probas={NEU: 0.603,...","AnalyzerOutput(output=anger, probas={anger: 0....",0.409091,0.391304,0.422222
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado po...",36,"AnalyzerOutput(output=NEU, probas={NEU: 0.524,...","AnalyzerOutput(output=others, probas={others: ...",1.000000,1.000000,1.000000
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.4.txt,o:mws-112-841,FP,FR.4,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"A Patiño. A el Nabuco de estos tiempos,á el qu...",25,"AnalyzerOutput(output=NEG, probas={NEG: 0.533,...","AnalyzerOutput(output=others, probas={others: ...",0.333333,0.333333,0.333333
1761-06-09_El-Duende-especulativo-sobre-la-vida-civil_Juan-Antonio-Mercadal-_Vol-1_Nr-01_093-235_FR.10.txt,o:mws-093-235,EX,FR.10,El-Duende-especulativo-sobre-la-vida-civil,Juan Antonio Mercadal [Francisco Mariano Nipho...,1761-06-09,1-01,"El chistoso Svvift pretendìa, que los Ingleses...",90,"AnalyzerOutput(output=NEU, probas={NEU: 0.680,...","AnalyzerOutput(output=others, probas={others: ...",0.333333,0.333333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.11.txt,o:mws.8083,FP,FR.11,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,“Mira. . . ; pero yo no podré mostrarte en est...,420,"AnalyzerOutput(output=NEG, probas={NEG: 0.900,...","AnalyzerOutput(output=anger, probas={anger: 0....",-0.027027,0.142857,-0.027027
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.3.txt,o:mws.8083,AE,FR.3,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,La más violenta devanadera en todo su ejercici...,2542,"AnalyzerOutput(output=NEU, probas={NEU: 0.589,...","AnalyzerOutput(output=others, probas={others: ...",0.052133,0.082569,0.056604
1812_El-Pensador-Mexicano_José-Joaquín-Fernández-de-Lizardi_Vol-1_Nr-011_8083_FR.5.txt,o:mws.8083,TR,FR.5,El-Pensador-Mexicano,José Joaquín Fernández de Lizardi,1812.0,1-011,Pero apenas el perezoso Morfeo había embargado...,1992,"AnalyzerOutput(output=NEG, probas={NEG: 0.625,...","AnalyzerOutput(output=others, probas={others: ...",0.043478,0.083333,0.049383


write emotion + nr of occurences in order, if no emotion is found write others

In [73]:
df["nf_emotion_lexicon_SAL"] = df["text"].progress_apply(compute_emotion)

  0%|          | 0/3053 [00:00<?, ?it/s]

In [74]:
df["lexicon_emotion"] = df["text"].progress_apply(one_emotion)

  0%|          | 0/3053 [00:00<?, ?it/s]

In [195]:
df

,issue pid,narrative form,xmlid,periodical title,author,year,issue number,text,nr_tokens,nf_sentiment_pysentimiento,nf_emotion_pysentimiento,nf_sentiment_lexicon_dispecs,nf_sentiment_lexicon_computational,nf_sentiment_lexicon_corrected,nf_emotion_lexicon_SAL,lexicon_emotion
filename,,,,,,,,,,,,,,,,
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-004_112-823_FR.3.txt,o:mws-112-823,S,FR.3,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-004,"En tànto que el Duende espèra á ver si purga el enfermo,ò si térco en sus maníasno obedece los remèdios: Por divertirse las Pascuas, como es alegre, i travieso,en el desván de los Duendesha puesto su Nacimiento. Lo mas especial que tiene, à más de ser todo nuebo,es que sacó de Paláciolas figúras, i los Gestos. Fue lo primero que hízo Portal del Palácio viejo,i Vino del CardenalArzobispo de Toledo. De su Padre el San Josef: que si no miente el comento,èl, tiene todas las señasde celos, i aun de recèlos. Virgen, no halló en el Palácio figúra que pueda serlo;i pidió prestada unaque serbía en un Convento. Del Patriarca la mula, por razon del mismo empleo;el Buey del Marques de Escòti,con licencia de su Dueño. De unas Camaristas, hízo los Angeles que digerongloria á Dios en las alturas,i páz al Hombre en el suelo. Nó se metió con las Damas; que el Duende es mui Caballero,i ni aún en chanza les faltaá las Dámas el respeto. No por que no àia matèria; que tiene el Palacio dentro,algunas Da...",274,"AnalyzerOutput(output=NEU, probas={NEU: 0.969, NEG: 0.017, POS: 0.014})","AnalyzerOutput(output=others, probas={others: 0.905, joy: 0.072, sadness: 0.009, surprise: 0.008, anger: 0.003, disgust: 0.001, fear: 0.001})",0.466667,0.647059,0.466667,"{'joy': 4, 'anger': 1, 'disgust': 1, 'fear': 0, 'sadness': 0, 'surprise': 0}",joy
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.12.txt,o:mws-112-841,F,FR.12,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Cuento de cuentos, historía sin verdad, i Fabula cierta. Erase, (que bien princípio) el mas opulento Reyno,que el Sol baña con sus luces,en uno, i otro Emisfério. Erase que estando pingue, famoso en armas, i hechos,en letras, i cristianidad,en indústria, i en consejos; De la noche à la mañana, todo caió por el suelo,sin saber como ni cuandoni por què: caso estupendo! Pues sus sencillos Vasallos no fueron la culpa de èsto,ni tampoco sus Magnates en nada de èsto incurrieron. Fué el caso que en el transcurso, que acordes causan los Tiemposvino a ser cabeza, un hombrepervertído, en el Gobierno. Este en lugar de escogerlos hombres de mas talentos,en quien descargar lo grabeque tràe con sígo este pèso,eligió de los mas rudos,humildes de nacimiento,déviles operaciones,i de intenciones perversos.Cargaron à los Vasallos,contribuciones, i pechos, I cada dia grabában de nuebo, todos los Pueblos.introdujose la Guerra,faltó el rocío del Cielo;i en médio de èste conflictolos aires se corrompiero...",696,"AnalyzerOutput(output=NEU, probas={NEU: 0.959, POS: 0.022, NEG: 0.020})","AnalyzerOutput(output=others, probas={others: 0.398, joy: 0.290, anger: 0.189, disgust: 0.047, surprise: 0.042, sadness: 0.029, fear: 0.005})",0.409091,0.391304,0.422222,"{'joy': 9, 'sadness': 6, 'disgust': 5, 'anger': 2, 'fear': 1, 'surprise': 1}",joy
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vol-1_Nr-010_112-841_FR.13.txt,o:mws-112-841,MT,FR.13,El-Duende-Crítico,Frai Manuel de San Josef,1735-1736,1-010,"Vele aquí el cuento acabado,aun que cortado por mèdio,que en punto de nobedades,lo dèjo para otro tiempo,como no buelba el buen Rojas,à pesquisarme los huesos;por que en èste caso, habráun Duende que le heche à perros.",36,"AnalyzerOutput(output=NEU, probas={NEU: 0.940, NEG: 0.056, POS: 0.004})","AnalyzerOutput(output=others, probas={others: 0.761, anger: 0.108, joy: 0.040, disgust: 0.030, sadness: 0.030, surprise: 0.024, fear: 0.007})",1.000000,1.000000,1.000000,others,others
1735-1736_El-Duende-Crítico_Frai-Manuel-de-San-Josef_Vo

### Saving the resulting dataframe

In [76]:
df.rename(columns = {'issue pid':'issue_pid','narrative form':'narrative_form','periodical title':'periodical_title','issue number': 'issue_number'}, inplace = True)


In [81]:
df.to_xml(f'{output}/NarrativeForms_emotion_sentiment.xml')

In [82]:
df.to_pickle(f'{output}/NarrativeForms_sentiment_emotion.pk')
